In [1]:
from Sindonews import Sindonews
import mysql.connector

In [2]:
obj = Sindonews()

In [4]:
links = obj.getAllBerita([], 1, 0, 0, '2018-08-02')

page  1
https://index.sindonews.com/index/0/0?t=2018-08-02
Insert berita  Densus 88 Antiteror Tangkap Tiga Terduga Teroris JAD di Gedebage
masuk
Insert berita  Wapres AS Sambut Jasad Tentara Korban Perang Korea
masuk
Insert berita  PN Sahkan Perdamaian Restrukturisasi Utang 5 Perusahaan Kemasan Plastik
masuk
Insert berita  Legislator Rusia Kecam Sanksi AS Terhadap Turki
masuk
Insert berita  Polres Jakbar Cokok Pemasok Sabu kepada Kelompok Jambret Jakarta
masuk
Insert berita  Bukopin Perluas Kerja Sama dengan Capital Life Indonesia
masuk
Insert berita  Israel Larang Bahan Bakar dan Gas Masuk ke Gaza
masuk
Insert berita  Venue Sepeda Veledrome Rawamangun Dipastikan Aman dari Kebakaran
masuk
Insert berita  Inggris Duduki Kursi Presiden Dewan Keamanan PBB
masuk
Insert berita  Industri Susu Segar Nasional Butuh Regulasi Lebih Tinggi
masuk
page  2
https://index.sindonews.com/index/0/10?t=2018-08-02
Insert berita  KPK Tengah Mengkaji Pidana Korporasi Kasus Suap PLTU Riau 1
masuk
Insert berita

masuk
Insert berita  Zikir di Istana Sukses, MDHW Ucapkan Terima Kasih ke Jokowi
masuk
Insert berita  Incar Pemilih Milenial, Prabowo Disarankan Lakukan Hal Ini
masuk
Insert berita  Pengembang Reklamasi dan Pemprov DKI Diminta Cari Solusi Sejahterakan Nelayan
masuk
Insert berita  IHSG Ditutup Jatuh 21,70 Poin, Bursa Asia Terkoreksi Perang Dagang
masuk
page  11
https://index.sindonews.com/index/0/100?t=2018-08-02
Insert berita  Jateng Runner Up Kejurnas Sepak Takraw Piala Menpora I
masuk
Insert berita  Bermasalah, AS Hancurkan Rudal Balistik Antarbenua Minuteman III
masuk
Insert berita  Javier Aguirre Jadi Pelatih Baru Timnas Mesir
masuk
Insert berita  Madonna Libatkan Ariana Grande Garap Album Baru
masuk
Insert berita  Gelapkan Dua Mobil, Dua Warga Grobogan Dipolisikan
masuk
Insert berita  Sopir Taksi Online Ditemukan Tewas di Sumedang, Ini Kronologisnya
masuk
Insert berita  MARS Sinergi dengan BPIP Terkait Pelestarian Budaya Nusantara
masuk
Insert berita  Diresmikan Jokowi, Bendungan 

masuk
Insert berita  Ciptakan Inovasi UKM, Kemenkop dan UKM Bangun Inkubator Bisnis
masuk
Insert berita  Masuk Mekkah Tanpa Visa Haji, 116 WNI Terjaring Razia
masuk
Insert berita  Privatisasi, Pemerintah Tawarkan Dua Bandara ke Investor
masuk
Insert berita  “Cinta yang Hilang”, Produksi MNC Pictures Tembus 100 Episode
masuk
Insert berita  Ganda Putra Jepang Bikin Fajar/Rian Angkat Koper
masuk
Insert berita  Performa Gemilang, Pemain Debutan Panen Pujian
masuk
Insert berita  Erdogan Tuduh AS Bermental Zionis dan Penginjil
masuk
page  21
https://index.sindonews.com/index/0/200?t=2018-08-02
Insert berita  Nafsu Memuncak, Seorang Remaja Menyelinap ke Kamar Gadis Tetangganya
masuk
Insert berita  Kovacic Gagal Terbang ke Manchester Gara-gara Mourinho
masuk
Insert berita  Mimpi Jani Terhenti di Gerbang KDI 2018
masuk
Insert berita  Korban Suhu Panas Diprediksi Naik 2.000%
masuk
Insert berita  Polisi Militer Tangkap 2 Jaringan Penipuan Tiket Pesawat di Bandara Juanda
masuk
Insert berita  Greys

masuk
Insert berita  Pemberlakukan Tarif 25% Bakal Memanaskan Perang Dagang AS-China
masuk
Insert berita  Permata Kerajaan Swedia Raib Digasak Pencuri
masuk
Insert berita  PKS Tetap Mendukung Rekomendasi Ijtimak Ulama
masuk
Insert berita  Pasca Pemilu, Kekerasan Pecah di Zimbabwe
masuk
Insert berita  Deflasi Terjadi di Kota Manado Sebesar 0,68%
masuk
Insert berita  CDM Filipina : Tim Bola Basket Kemungkinan Tampil di Asian Games 2018
masuk
Insert berita  Jenderal Iran: Kemampuan Perang dan Moral Iran Terlalu Besar untuk AS
masuk
Insert berita  Deklarasi SAS Institute, Jadikan Islam Nusantara sebagai Pemersatu Bangsa
masuk
Insert berita  Nissan Kicks Resmi Gantikan Juke di Amerika Serikat
masuk
page  31
https://index.sindonews.com/index/0/300?t=2018-08-02
Insert berita  Gerindra Tak Yakin PKS Bakal Abstain di Pilpres
masuk
Insert berita  Dorong Konsumsi Ikan, Rekor Muri Bakso Ikan SNI Akan Digelar
masuk
Insert berita  'Digebuk' Taliban, 200 Lebih Anggota ISIS Menyerahkan Diri
masuk
Inse

KeyError: 'data-ci-pagination-page'

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from requests.exceptions import ConnectionError
import unicodedata
import mysql.connector

In [ ]:
articles = {}

In [ ]:
url = "https://index.sindonews.com/index/0/20?t=2018-08-04"

In [ ]:
response = requests.get(url)
# Extract HTML texts contained in Response object: html
html = response.text
# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [ ]:
el_page = soup.find('div', class_="pagination")
el_page
# if el_page:
max_page = el_page.findAll('a')[-1]['data-ci-pagination-page']
# max_page
active_page = el_page.find('li', class_="active").get_text(strip=True)
active_page
